In [1]:
import pickle
import pandas as pd
import itertools
import copy as cp
from collections import Counter
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string
nltk.download('stopwords')
from tqdm.auto import tqdm
from datasets import load_dataset
dataset = load_dataset("LYTinn/sentiment-analysis-tweet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maenz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Found cached dataset csv (C:/Users/maenz/.cache/huggingface/datasets/LYTinn___csv/LYTinn--sentiment-analysis-tweet-084cc9bd3eb759fe/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tweet', 'label'],
        num_rows: 4913
    })
})

In [3]:
df=pd.DataFrame.from_dict(dataset["train"][0:700])

In [4]:
df.iloc[0:5]['tweet']

0    BofA previews Netflixs NFLX Q3 Earnings Tues 0...
1    I scooped a couple of shares this morning at a...
2    Im streaming ES Futures using Bookmap on youtu...
3               CF taking some off here close to 19150
4    No change to this position is still bullish st...
Name: tweet, dtype: object

In [5]:
df_train=df

In [6]:
df_test=pd.DataFrame.from_dict(dataset["test"][0:300])

In [7]:
for index, row in df_train.iterrows():
    if row['label']==-1:
        df_train.at[index,'label']=2
for index, row in df_test.iterrows():
    if row['label']==-1:
        df_test.at[index,'label']=2

In [8]:
# df_train.to_csv("tweet_train")
# df_test.to_csv("tweet_test")

In [9]:
df_train=pd.read_csv("tweet_train")
df_text=pd.read_csv("tweet_test")

In [10]:
X_train_original=df_train['tweet']
X_valid_original=df_test['tweet']
y_train_original=df_train['label']
y_valid_original=df_test['label']

In [11]:
import torch
assert torch.cuda.is_available()
device = torch.device("cuda")

In [12]:
from transformers import AutoTokenizer, AdamW,BertForSequenceClassification
tokenizer= AutoTokenizer.from_pretrained("bert-base-uncased")
X_train=tokenizer(list(X_train_original),truncation=True, padding=True, max_length=32,return_tensors='pt')
y_train=list(y_train_original)
X_valid=tokenizer(list(X_valid_original),truncation=True, padding=True, max_length=32,return_tensors='pt')
y_valid=list(y_valid_original)

In [13]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, token):
        self.token = token
    def __getitem__(self, id):
        return {key: val[id] for key, val in self.token.items()}
    def __len__(self):
        return len(self.token.input_ids)

In [14]:
bat_size=64
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)
valid_dataloader = DataLoader(MyData(X_valid) , batch_size=bat_size,shuffle = False)

In [15]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)
    
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

epoch:   0%|          | 0/50 [00:00<?, ?it/s]

trainset:   0%|          | 0/11 [00:00<?, ?it/s]

validset:   0%|          | 0/5 [00:00<?, ?it/s]

epoch: 0
train loss: 1.047587351365523
valid loss: 1.0729750871658326
valid accuracy: 0.45
——————————————————————————————————————————————


trainset:   0%|          | 0/11 [00:00<?, ?it/s]

...

validset:   0%|          | 0/5 [00:00<?, ?it/s]

epoch: 21
train loss: 0.3880479633808136
valid loss: 1.3935454130172729
valid accuracy: 0.5266666666666666
——————————————————————————————————————————————


trainset:   0%|          | 0/11 [00:00<?, ?it/s]

validset:   0%|          | 0/5 [00:00<?, ?it/s]

early stop


In [17]:
print(max(valid_accuracy_list))

0.55


## baseline

In [18]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentence: " +text}])
    return completion.choices[0].message.content

In [19]:
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             ans=rephrase_by_chatgpt(X_train_original[i])
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)

In [20]:
# base_dict={"tweet":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("chatgpt_tweet_base")
new_df=pd.read_csv("chatgpt_tweet_base")
X_train_base=list(X_train_original)+list(new_df['tweet'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=32,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [21]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/22 [00:00<…

validset:   0%|                                                                                  | 0/5 [00:00<…

epoch: 0
train loss: 1.1965433196587996
valid loss: 1.1531194210052491
valid accuracy: 0.45
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/22 [00:00<…

...

validset:   0%|                                                                                  | 0/5 [00:00<…

epoch: 17
train loss: 0.4852109036662362
valid loss: 1.1406179189682006
valid accuracy: 0.5266666666666666
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/22 [00:00<…

validset:   0%|                                                                                  | 0/5 [00:00<…

early stop


In [23]:
print(max(valid_accuracy_list))

0.5866666666666667


### +label

In [41]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt2(text,label):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    if label==1:
        tone="positive"
    elif label==2:
        tone="negative"
    else:
        tone="neutral"
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the sentence while retaining  the"+ tone + "emotion: "+text}])
    return completion.choices[0].message.content

In [42]:
rephrase_by_chatgpt2(X_train_original[7],y_train_original[7])

'The satisfaction of causing harm has been achieved, but perhaps a glimmer of hope exists that is yet to be uncovered. Consumers should exercise caution.'

In [43]:
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            ans=rephrase_by_chatgpt2(X_train_original[i],y_train_original[i])
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1  
        except:
            time.sleep(0.5)

  0%|                                                                                          | 0/700 [00:00<…

In [44]:
base_dict={"tweet":x_new,"label":y_new}
new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_tweet_label")
new_df=pd.read_csv("chatgpt_tweet_label")
X_train_base=list(X_train_original)+list(new_df['tweet'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=32,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [45]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [46]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/22 [00:00<…

validset:   0%|                                                                                  | 0/5 [00:00<…

epoch: 0
train loss: 1.1467652131210675
valid loss: 1.1265252351760864
valid accuracy: 0.45
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/22 [00:00<…

...

validset:   0%|                                                                                  | 0/5 [00:00<…

epoch: 25
train loss: 0.4362465366721153
valid loss: 1.2348727703094482
valid accuracy: 0.5633333333333334
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/22 [00:00<…

validset:   0%|                                                                                  | 0/5 [00:00<…

early stop


In [47]:
print(max(valid_accuracy_list))

0.6133333333333333


### back translation

In [21]:
import os
import openai
import time
from auto_tqdm import tqdm
def chatgpt_to_spain(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please translate this sentence to Spanish while retaining emotion:"+text}])
    return completion.choices[0].message.content
def chatgpt_to_english(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please translate this sentence to English:"+text}])
    return completion.choices[0].message.content

In [ ]:
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            temp=chatgpt_to_spain(X_train_original[i])
            ans=chatgpt_to_english(temp)
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1  
        except:
            time.sleep(0.5)

  0%|                                                                                          | 0/700 [00:00<…

In [ ]:
# base_dict={"text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("twitter_back_translate")
new_df=pd.read_csv("twitter_back_translate")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=32,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [ ]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

In [ ]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

In [ ]:
print(max(valid_accuracy_list))

## p2

In [22]:
# import os
# import openai
# import time
# from auto_tqdm import tqdm
# def rephrase_by_chatgptp2(text):
#     file1=open("api_key.txt",'r')
#     api_key=file1.read()
#     openai.api_key=api_key
#     completion = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo",
#     messages=[{'role': 'user', 'content': "Please rephrase the following sentences from poem but keep their emotion " +text}])
#     return completion.choices[0].message.content
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             ans=rephrase_by_chatgptp2(X_train_original[i])
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)
# base_dict={"verse_text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_twitter_p2")
new_df=pd.read_csv("chatgpt_twitter_p2")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=32,return_tensors='pt')
y_train=list(y_train_base)

In [23]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/22 [00:00<…

validset:   0%|                                                                                  | 0/5 [00:00<…

epoch: 0
train loss: 1.03231572562998
valid loss: 1.0426148414611816
valid accuracy: 0.45
——————————————————————————————————————————————


...

validset:   0%|                                                                                  | 0/5 [00:00<…

epoch: 19
train loss: 0.5058319229971279
valid loss: 1.2835144758224488
valid accuracy: 0.51
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/22 [00:00<…

validset:   0%|                                                                                  | 0/5 [00:00<…

early stop


In [24]:
print(max(valid_accuracy_list))

0.5766666666666667


## p3

In [34]:
# import os
# import openai
# import time
# from auto_tqdm import tqdm
# def rephrase_by_chatgptp3(text):
#     file1=open("api_key.txt",'r')
#     api_key=file1.read()
#     openai.api_key=api_key
#     completion = openai.ChatCompletion.create(
#     model="gpt-3.5-turbo",
#     messages=[{'role': 'user', 'content': "Please rephrase the following sentences to around 12 length words without changing emotion: " + text}])
#     return completion.choices[0].message.content
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             ans=rephrase_by_chatgptp3(X_train_original[i])
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)
# base_dict={"text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_twitter_p3")
new_df=pd.read_csv("chatgpt_twitter_p3")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=32,return_tensors='pt')
y_train=list(y_train_base)

In [35]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 4,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/22 [00:00<…

validset:   0%|                                                                                  | 0/5 [00:00<…

epoch: 0
train loss: 1.1599074656313115
valid loss: 1.1119572401046753
valid accuracy: 0.45666666666666667
——————————————————————————————————————————————


...

validset:   0%|                                                                                  | 0/5 [00:00<…

epoch: 16
train loss: 0.6258524385365573
valid loss: 1.1205647110939025
valid accuracy: 0.5433333333333333
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/22 [00:00<…

validset:   0%|                                                                                  | 0/5 [00:00<…

early stop


In [36]:
print(max(valid_accuracy_list))

0.5666666666666667
